In [14]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["MCA_WT"]

# Define collections
movies_collection = db["movies"]
actors_collection = db["actors"]
producers_collection = db["producers"]

# Drop collections if they exist for a clean start
movies_collection.drop()
actors_collection.drop()
producers_collection.drop()

# Sample Data for Movies
movies_data = [
    {
        "_id": 1,
        "title": "Action Thriller",
        "budget": 5000000,
        "release_year": 2022,
        "actors": [{"name": "Akshay", "role": "Lead"}, {"name": "Rohit", "role": "Supporting"}],
        "producers": ["Rajesh", "Vikram"]
    },
    {
        "_id": 2,
        "title": "Romantic Comedy",
        "budget": 3000000,
        "release_year": 2021,
        "actors": [{"name": "Akshay", "role": "Lead"}, {"name": "Priya", "role": "Supporting"}],
        "producers": ["Rajesh"]
    },
    {
        "_id": 3,
        "title": "Drama Story",
        "budget": 7000000,
        "release_year": 2023,
        "actors": [{"name": "Rohit", "role": "Lead"}, {"name": "Akshay", "role": "Supporting"}],
        "producers": ["Vikram", "Sonal","Rajesh"]
    },
    {
        "_id": 4,
        "title": "Horror Nights",
        "budget": 4000000,
        "release_year": 2022,
        "actors": [{"name": "Priya", "role": "Lead"}, {"name": "Akshay", "role": "Supporting"}],
        "producers": ["Sonal"]
    },
    {
        "_id": 5,
        "title": "Sci-Fi Adventure",
        "budget": 8000000,
        "release_year": 2023,
        "actors": [{"name": "Rohit", "role": "Lead"}, {"name": "Priya", "role": "Supporting"}],
        "producers": ["Rajesh", "Sonal","Rajesh","Rajesh"]
    }
]

# Sample Data for Actors
actors_data = [
    {"_id": 1, "name": "Akshay", "movies": ["Action Thriller", "Romantic Comedy", "Drama Story", "Horror Nights"]},
    {"_id": 2, "name": "Rohit", "movies": ["Action Thriller", "Drama Story", "Sci-Fi Adventure"]},
    {"_id": 3, "name": "Priya", "movies": ["Romantic Comedy", "Horror Nights", "Sci-Fi Adventure"]}
]

# Sample Data for Producers
producers_data = [
    {"_id": 1, "name": "Rajesh", "movies": ["Action Thriller", "Romantic Comedy", "Sci-Fi Adventure"]},
    {"_id": 2, "name": "Vikram", "movies": ["Action Thriller", "Drama Story"]},
    {"_id": 3, "name": "Sonal", "movies": ["Drama Story", "Horror Nights", "Sci-Fi Adventure"]}
]

# Insert Data into Collections
movies_collection.insert_many(movies_data)
actors_collection.insert_many(actors_data)
producers_collection.insert_many(producers_data)


InsertManyResult([1, 2, 3], acknowledged=True)

In [15]:
# Query (a): List the names of movies with the highest budget.

# Find the movie(s) with the highest budget
highest_budget_movie = movies_collection.find_one(sort=[("budget", -1)])
print(highest_budget_movie)
highest_budget_movie = highest_budget_movie["budget"]
print(highest_budget_movie)

highest_budget_movies = list(movies_collection.find({"budget": highest_budget_movie}, {"title": 1, "_id": 0}))
for movie in highest_budget_movies:
    print(movie["title"])

{'_id': 5, 'title': 'Sci-Fi Adventure', 'budget': 8000000, 'release_year': 2023, 'actors': [{'name': 'Rohit', 'role': 'Lead'}, {'name': 'Priya', 'role': 'Supporting'}], 'producers': ['Rajesh', 'Sonal', 'Rajesh', 'Rajesh']}
8000000
Sci-Fi Adventure


In [16]:
# Query (b): Display the details of producers who have produced more than one movie in a year.

# Find producers who have produced more than one movie
producers_with_multiple_movies = list(producers_collection.aggregate([
    {"$unwind": "$movies"},
    # group the documents by the name field
    # "movie_count": {"$sum": 1}: This calculates the number of movies for each producer 
    # by summing up 1 for each document (or "movie") that belongs to a particular name.
    {"$group": {"_id": "$name", "movie_count": {"$sum": 1}}},
    {"$match": {"movie_count": {"$gt": 1}}}
]))

for producer in producers_with_multiple_movies:
    print(producer)


# before unwind
# {
#     "_id": 1,
#     "name": "Rajesh",
#     "movies": ["Action Thriller", "Romantic Comedy", "Sci-Fi Adventure"]
# }
# {
#     "_id": 2,
#     "name": "Akhilesh",
#     "movies": [ "Comedy", "Adventure"]
# }


# after unwind
# [
#     { "_id": 1, "name": "Rajesh", "movies": "Action Thriller" },
#     { "_id": 1, "name": "Rajesh", "movies": "Romantic Comedy" },
#     { "_id": 1, "name": "Rajesh", "movies": "Sci-Fi Adventure" },
#     { "_id": 2, "name": "Rajesh", "movies": "Comedy" },
#     { "_id": 2, "name": "Rajesh", "movies": "Adventure" },
# ]


{'_id': 'Vikram', 'movie_count': 2}
{'_id': 'Sonal', 'movie_count': 3}
{'_id': 'Rajesh', 'movie_count': 3}


In [17]:
# Query (c): List the names of actors who have acted in at least one movie, in which "Akshay" has acted.

# Find movies where "Akshay" has acted
akshay_movies = actors_collection.find_one({"name": "Akshay"}, {"movies": 1})["movies"]

# Find all actors who have acted in at least one of these movies
actors_in_akshay_movies = list(actors_collection.find({"movies": {"$in": akshay_movies}}, {"name": 1, "_id": 0}))

# Exclude Akshay's name
actors_in_akshay_movies = [actor["name"] for actor in actors_in_akshay_movies if actor["name"] != "Akshay"]
print(actors_in_akshay_movies)

['Rohit', 'Priya']


In [22]:
# Query (d): List the names of movies produced by more than one producer.

# Find movies with more than one producer
movies_with_multiple_producers = list(movies_collection.find({"producers.1": {"$exists": True}}, {"title": 1, "_id": 0}))
for movie in movies_with_multiple_producers:
    print(movie["title"])

# m = movies_collection.find({},{"title":1,"producers":1,"_id":0})
# for movie in m:
#     if len(movie["producers"]) > 2:
#         print(movie['title'])

Action Thriller
Drama Story
Sci-Fi Adventure
